In [1]:
%cd NYSE-Temporal-Graph-Construction

[Errno 2] No such file or directory: 'NYSE-Temporal-Graph-Construction'
/teamspace/studios/this_studio


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
import networkx as nx
from matplotlib import pyplot as plt
from tqdm import tqdm
from os.path import join as join_path
import os
from torch.utils.data import Dataset
import kagglehub
import numpy as np
import pandas as pd
from functools import cache
import torch
from torch_geometric_temporal.nn.recurrent import GConvGRU, GCLSTM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cpu


In [3]:
TARGET_SECTORS = ['Information Technology']
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1

In [4]:
dataset_path = kagglehub.dataset_download("dgawlik/nyse")

In [5]:
fundamentals_df = pd.read_csv(join_path(dataset_path,"fundamentals.csv"))
df = pd.read_csv(join_path(dataset_path, "prices.csv"))
prices_split_adjusted_df = pd.read_csv(join_path(dataset_path, "prices-split-adjusted.csv"))
securities_df = pd.read_csv(join_path(dataset_path, "securities.csv"))

In [6]:
all_symbols = prices_split_adjusted_df['symbol'].sort_values().unique()

def get_symbol_by_sector(sector):
    return securities_df[securities_df['GICS Sector'] == sector]['Ticker symbol'].to_list()

target_symbols = prices_split_adjusted_df['symbol'].sort_values().unique()

target_symbols = []

if not TARGET_SECTORS or len(TARGET_SECTORS) == 0:
    target_symbols = all_symbols.tolist()
else:
    for sector in TARGET_SECTORS:
        target_symbols += get_symbol_by_sector(sector)

target_symbols.sort()
    
@cache
def symbol_to_int(symbol: str):
    if symbol not in target_symbols:
        return -1
    return target_symbols.index(symbol)

@cache
def int_to_symbol(idx):
    idx = int(idx)
    return target_symbols[idx]


print("Num of all symbols:", len(all_symbols))
print("Num of target symbols:", len(target_symbols), target_symbols)


Num of all symbols: 501
Num of target symbols: 68 ['AAPL', 'ACN', 'ADBE', 'ADI', 'ADP', 'ADS', 'ADSK', 'AKAM', 'AMAT', 'APH', 'ATVI', 'AVGO', 'CA', 'CRM', 'CSCO', 'CSRA', 'CTSH', 'CTXS', 'EA', 'EBAY', 'FB', 'FFIV', 'FIS', 'FISV', 'FLIR', 'FSLR', 'GLW', 'GOOG', 'GOOGL', 'GPN', 'HPE', 'HPQ', 'HRS', 'IBM', 'INTC', 'INTU', 'JNPR', 'KLAC', 'LLTC', 'LRCX', 'MA', 'MCHP', 'MSFT', 'MSI', 'MU', 'NFLX', 'NTAP', 'NVDA', 'ORCL', 'PAYX', 'PYPL', 'QCOM', 'QRVO', 'RHT', 'STX', 'SWKS', 'SYMC', 'TDC', 'TEL', 'TSS', 'TXN', 'V', 'VRSN', 'WDC', 'WU', 'XLNX', 'XRX', 'YHOO']


In [7]:
prices_split_adjusted_df['date'] = pd.to_datetime(prices_split_adjusted_df['date']).dt.date
dates = prices_split_adjusted_df['date'].sort_values().unique()


def any_to_date(date):
    if not isinstance(date, pd._libs.tslibs.timestamps.Timestamp):
        date = pd.to_datetime(date).date()
    return date

@cache
def date_to_int(date):
    date = any_to_date(date)
    return dates.tolist().index(date)

@cache
def int_to_date(idx):
    return dates[idx]

print(dates)

[datetime.date(2010, 1, 4) datetime.date(2010, 1, 5)
 datetime.date(2010, 1, 6) ... datetime.date(2016, 12, 28)
 datetime.date(2016, 12, 29) datetime.date(2016, 12, 30)]


In [8]:

# class NYSEDailyDataset(Dataset):
#     def __init__(self, prices_df):
#         self.df = prices_df.copy()
        
#         self.df['date'] = self.df['date'].apply(date_to_int)
#         self.df['symbol'] = self.df['symbol'].apply(symbol_to_int)

#         self.df.sort_values(['date', 'symbol'], inplace=True)


#         print("df", self.df.head(10))

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
        
#         return self.df.iloc[idx].to_numpy().tolist()
        


# dataset = NYSEDailyDataset(prices_split_adjusted_df[:500])

In [130]:
def load_data(target_df, fill_missing=True, repeat=1):

    df = target_df.copy()
    
    df['date'] = df['date'].apply(date_to_int)
    df['symbol'] = df['symbol'].apply(symbol_to_int)

    # remove symbol == -1
    df = df[df['symbol'] != -1]

    df.sort_values(['date', 'symbol'], inplace=True)


    for date, group in df.groupby('date'):
        group.drop('date', axis=1, inplace=True)
        group.set_index('symbol', inplace=True)
        if fill_missing:
            group = group.reindex(range(len(target_symbols)), fill_value=0.0)
            
        if repeat > 1:
            for _ in range(repeat):
                yield date, group
        else:
            yield date, group


In [132]:
for date, batch in load_data(prices_split_adjusted_df.sort_values('date').head(500)):
    print(date, batch.to_numpy().shape)
    

batched_data = dict(load_data(prices_split_adjusted_df.sort_values('date')))

0 (68, 5)
1 (68, 5)


In [146]:
# # class Attention(torch.nn.Module):
# #     def __init__(self, input_dim, head_dim=1, num_heads=len(target_symbols), device=device, normalize=False, verbose=False):
# #         super().__init__()

# #         self.input_dim = input_dim
# #         self.hidden_dim = head_dim * num_heads
# #         self.num_heads = num_heads
# #         self.head_dim = head_dim
        

# #         self.query = torch.nn.Linear(input_dim, self.hidden_dim)
# #         self.key = torch.nn.Linear(input_dim, self.hidden_dim)
        
# #         self.scale = torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32)).to(device)

# #         self.softmax = torch.nn.Softmax(dim=-1)
# #         self.normalize = normalize
# #         self.verbose = verbose

# #         torch.nn.init.xavier_uniform_(self.query.weight)
# #         torch.nn.init.xavier_uniform_(self.key.weight)
    
# #     def forward(self, x):
# #         seq_len, feat_dim = x.shape
# #         # Ensure `self.scale` is on the same device as `x`
# #         # self.scale = self.scale.to(x.device)
        
# #         # Compute queries and keys
# #         q = self.query(x)
# #         k = self.key(x)
        
# #         if self.verbose:
# #             print("seq_len", seq_len)
# #             print("Q",q.shape, q.min(), q.max())
# #             print("K",q.shape, k.min(), k.max())
        
# #         q = q.view(seq_len, self.num_heads, self.head_dim).transpose(0, 1)  # Shape: (batch_size, num_heads, seq_len, head_dim)
# #         k = k.view(seq_len, self.num_heads, self.head_dim).transpose(0, 1)  # Shape: (batch_size, num_heads, seq_len, head_dim)

# #         if self.verbose:
# #             print("Q split",q.shape)
# #             print("K split",k.shape)        

# #         # Dot-product attention
# #         attn = torch.bmm(q, k.transpose(-2, -1)) / self.scale  # Scaling only once
        
# #         if self.verbose:
# #             print("Raw Atts", attn)

# #         if self.normalize:
# #             attn = attn / torch.norm(attn, dim=-1, keepdim=True)
# #         else:
# #             # Apply softmax to normalize scores
# #             attn = self.softmax(attn)

# #         # Set diagonal to 0
# #         attn = attn.masked_fill(torch.eye(attn.shape[0], dtype=torch.bool), 0)
        
# #         return attn


class GraphConstructor(torch.nn.Module):

    def __init__(self, input_dim, num_nodes=len(target_symbols), temp=1.0):
        super(GraphConstructor, self).__init__()

        self.input_dim = input_dim
        self.num_nodes = num_nodes
        self.temp = temp

        self.q = torch.nn.Linear(input_dim, num_nodes)
        self.k = torch.nn.Linear(input_dim, num_nodes)
        self.v = torch.nn.Linear(input_dim, num_nodes)

        self.att = torch.nn.MultiheadAttention(num_nodes, num_heads=1)


    def forward(self, x):
        Q = self.q(x)
        K = self.k(x)
        V = self.v(x)

        output, attention_weights = self.att(Q,K,V)
        adj = output
        adj = torch.nn.functional.sigmoid(output/self.temp)
        # set diameter to 0
        adj = adj.masked_fill(torch.eye(adj.shape[0], dtype=torch.bool), 0)
        return adj


# constructor = Attention(input_dim=5, head_dim=2, normalize=False, verbose=False).to(device)
constructor = GraphConstructor(input_dim=5).to(device)
torch.set_printoptions(profile="full")
d = torch.tensor(batched_data[0].to_numpy(), dtype=torch.float).to(device)
adj_matrix = constructor(d)
print(adj_matrix.shape, adj_matrix)

# # class Predictor(torch.nn.Module):
# #     def __init__(self, input_dim, att_dim, hidden_dim):
# #         super().__init__()
# #         self.graph_constructor = GraphConstructor(input_dim, att_dim)
# #         self.recurrent = GConvGRU(input_dim, embed_dim, 10)
# #         self.ffn = torch.nn.Sequential(
# #             torch.nn.Linear(att_dim, 64),
# #             torch.nn.ReLU(),
# #             torch.nn.Linear(64, 32),
# #             torch.nn.ReLU(),
# #             torch.nn.Linear(32, hidden_dim),
# #             torch.nn.ReLU()
# #         )
        
# #         self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=3, batch_first=True)

# #     def forward(self, x):
        
# #         att_out = self.graph_constructor(x)

# #         ffn_out = self.ffn(att_out)

# #         out, _ = self.rnn(ffn_out)

# #         return out


# # myNN = Predictor(input_dim=6, att_dim=64, hidden_dim=32).to(device)

model  = constructor

torch.Size([68, 68]) tensor([[0.0000, 0.4779, 0.5159, 0.5151, 0.4853, 0.4571, 0.5085, 0.4998, 0.5324,
         0.5076, 0.5188, 0.4755, 0.5134, 0.4637, 0.5126, 0.4733, 0.5413, 0.4965,
         0.5277, 0.5270, 0.4945, 0.5619, 0.5106, 0.4586, 0.4668, 0.5023, 0.4953,
         0.4730, 0.4865, 0.4778, 0.5299, 0.4768, 0.5041, 0.4708, 0.4651, 0.4673,
         0.4924, 0.4956, 0.5053, 0.5258, 0.5208, 0.4929, 0.5142, 0.4738, 0.4695,
         0.4583, 0.4931, 0.4866, 0.5019, 0.5297, 0.5033, 0.5327, 0.5084, 0.5191,
         0.4898, 0.5032, 0.4978, 0.4875, 0.4744, 0.5320, 0.4944, 0.4891, 0.5260,
         0.4840, 0.5145, 0.4800, 0.4922, 0.4976],
        [0.5037, 0.0000, 0.5159, 0.5151, 0.4853, 0.4571, 0.5085, 0.4998, 0.5324,
         0.5076, 0.5188, 0.4755, 0.5134, 0.4637, 0.5126, 0.4733, 0.5413, 0.4965,
         0.5277, 0.5270, 0.4945, 0.5619, 0.5106, 0.4586, 0.4668, 0.5023, 0.4953,
         0.4730, 0.4865, 0.4778, 0.5299, 0.4768, 0.5041, 0.4708, 0.4651, 0.4673,
         0.4924, 0.4956, 0.5053, 0.525

In [143]:
def random_adj_matrix(size):
    a = torch.rand(size, size, device=device)
    # Set diamter to zero
    for i in range(size):
        a[i, i] = 0
    
    return a

def random_unweighted(size):
    a = random_adj_matrix(size)
    # Set weights to 1 if greeated than 0.5
    a[a >= 0.5] = 1
    a[a < 0.5] = 0
    return a


def fully_connected(size):
    a = torch.ones(size, size, device=device)
    for i in range(size):
        a[i, i] = 0
    return a

print(random_adj_matrix(5))
print(random_unweighted(5))
print(fully_connected(5))

tensor([[0.0000, 0.0192, 0.2229, 0.1259, 0.3523],
        [0.1935, 0.0000, 0.0669, 0.4388, 0.0223],
        [0.5593, 0.4562, 0.0000, 0.4178, 0.5414],
        [0.4217, 0.4275, 0.1659, 0.0000, 0.6158],
        [0.9529, 0.0117, 0.3945, 0.2837, 0.0000]])
tensor([[0., 1., 1., 1., 0.],
        [1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1.],
        [1., 1., 0., 1., 1.],
        [1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 0.]])


In [148]:
# model = Attention(input_size=7, hidden_size=64).to(device)
true_graph = random_unweighted(len(target_symbols))

def train(model, epochs=100, true_graph=true_graph):
    print("True graph", true_graph[0][0:10])

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(epochs):
        total_loss = 0
        model.train(True)
        pbar = tqdm(range(len(dates)), desc="Epoch {}".format(epoch + 1))
        pbar.clear()
        for i, (date, date_data) in enumerate(load_data(prices_split_adjusted_df.sort_values('date'), repeat=24)):
            optimizer.zero_grad()
            date_data = torch.tensor(date_data.to_numpy(), dtype=torch.float).to(device)
            pred = model(date_data)

            loss = criterion(pred, true_graph)
            
            # if i % (24*100) == 0:
            print(i, "pred", pred[0][0:10], "loss", loss)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

train(model)


True graph tensor([0., 1., 0., 1., 0., 0., 1., 0., 1., 1.])


0 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
1 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
2 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
3 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
4 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
5 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
6 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss tensor(0.4913, grad_fn=<MseLossBackward0>)
7 pred tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>) loss ten

KeyboardInterrupt: 